In [24]:
# In[1]:


import pandas as pd
import numpy as np
import math
from collections import Counter
import keras

# In[2]:

sensor_data = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
Counter(sensor_data.TagName)


# In[3]:


sensor_data_acc = sensor_data[sensor_data.SENSORTYPE == 1]
sensor_data_acc_number_tag = sensor_data_acc[~((sensor_data_acc.TagName == 'Tag0') | (sensor_data_acc.TagName == 'Start1') | (sensor_data_acc.TagName == 'Start2'))]


# In[4]:

Counter(sensor_data_acc_number_tag.TagName)


# In[5]:

# tag_id_groupby = sensor_data_acc_tag12.groupby(['TagName', 'tester_id'])
tag_id_groupby = sensor_data_acc_number_tag.groupby(['TagName', 'tester_id'])
tag_id_dict = tag_id_groupby.groups
y = list(tag_id_dict.keys())
y = [i[0] for i in y]
y = np.array(y)


# In[6]:

y.shape


# In[7]:

# there will be 9 frames, frame0 to frame8
# There will be 10 segments
N_frame_no = 9 

test_percent = 0.1 # 10% samples are used for testing


# In[8]:

def gesture_features(accs):
    Ls = math.floor(len(accs)/ (N_frame_no + 1))
    segments = None
    for i in range(N_frame_no + 1):
        if segments is None:
            segments = np.array([accs[i*Ls:(i+1)*Ls]])
        else:
            segments = np.append(segments, np.array([accs[i*Ls:(i+1)*Ls]]), axis=0)

    frames = None
    for i in range(N_frame_no):
        cur_frame = segments[i:i+2]
        cur_frame = cur_frame.reshape((cur_frame.shape[0]*cur_frame.shape[1],cur_frame.shape[2]))
        if frames is None:
            frames = np.array([cur_frame])
        else:
            frames = np.append(frames, np.array([cur_frame]), axis = 0)
    return np.array([frame_features(f,Ls) for f in frames])


# In[9]:

def frame_features(cur_frame, Ls):
    dft_cur_frame = np.fft.fftn(cur_frame)
    
    mean_cur_frame = np.array([dft_cur_frame[0]])

    energy_cur_frame=[]
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        for i in range(1,Ls*2):
            T_sum += math.pow(abs(dft_cur_frame[i,T]),2)

        energy_cur_frame.append(T_sum / abs(Ls*2-1))
    energy_cur_frame = np.array([energy_cur_frame])
    
    entropy_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        entropy_sum = 0
        for i in range(1,Ls*2):
            T_sum += abs(dft_cur_frame[i,T])

        for m in range(1,Ls*2):
            p_m_T = abs(dft_cur_frame[m,T])/T_sum
            entropy_sum += p_m_T*math.log(1/p_m_T)
        entropy_cur_frame.append(entropy_sum)
    entropy_cur_frame = np.array([entropy_cur_frame])
    
    std_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        std_cur_frame.append(np.std(cur_frame))
    std_cur_frame = np.array([std_cur_frame])
    
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    # correlation
    
    return np.concatenate((mean_cur_frame,energy_cur_frame,std_cur_frame,entropy_cur_frame))


# In[ ]:




# In[10]:

X = []
for key in list(tag_id_dict.keys()):
    X.append(gesture_features(tag_id_groupby.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values))
X = np.array(X)


# In[11]:

X.shape


(963, 9, 4, 3)

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Input, Dense
from keras.models import Model

In [5]:
cloned_y = np.zeros(y.shape)
for num, tag_str in list(enumerate(set(y))):
    cloned_y[y==tag_str] = num

In [6]:

y_categorical = keras.utils.to_categorical(cloned_y, num_classes=9)

In [74]:
def normalize_feature(feature):
    maxi = max(feature.reshape(-1))
    mini = min(feature.reshape(-1))
    return (feature - mini) / (maxi - mini)

In [75]:
# X.shape (..., 9, 4, 3)
normalized_X = None
for feature_idx in range(X.shape[2]):
    cur_normed = normalize_feature(X[:,:,feature_idx,:]).reshape((len(X), X.shape[1]*X.shape[3]))
    if normalized_X is None:
        normalized_X = cur_normed
    else:
        normalized_X = np.append(normalized_X, cur_normed, axis = 1)

In [76]:
normalized_X.shape

(963, 108)

In [79]:
idx_list = list(range(y.size))
np.random.shuffle(idx_list)
train_idx = idx_list[:math.floor(len(X) * (1- test_percent))]
test_idx = idx_list[math.floor(len(X) * (1- test_percent)):]


train_x = normalized_X[train_idx]
train_y = y_categorical[train_idx]
test_x = normalized_X[test_idx]
test_y = y_categorical[test_idx]



# This returns a tensor
inputs = Input(shape=(normalized_X.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
layer1 = Dense(64, activation='relu')(inputs)
layer2 = Dense(128, activation='relu')(layer1)
layer3 = Dense(64, activation='relu')(layer2)
layer4 = Dense(32, activation='relu')(layer3)
predictions = Dense(y_categorical.shape[-1], activation='softmax')(layer4)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_his = model.fit(train_x, train_y, batch_size=32, epochs=40, verbose = 0)  # starts training
pred_y = model.predict(test_x)
result_label = np.argmax(pred_y, 1)
print(classification_report(cloned_y[test_idx],result_label))
print(confusion_matrix(cloned_y[test_idx],result_label))

/Users/Ageha/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


             precision    recall  f1-score   support

        0.0       0.25      0.11      0.15         9
        1.0       0.50      0.22      0.31         9
        2.0       0.15      0.88      0.25         8
        3.0       1.00      0.36      0.53        14
        4.0       0.75      0.38      0.50         8
        5.0       0.00      0.00      0.00        10
        6.0       0.00      0.00      0.00        18
        7.0       0.33      0.92      0.49        12
        8.0       0.00      0.00      0.00         9

avg / total       0.33      0.30      0.24        97

[[ 1  0  2  0  0  0  0  6  0]
 [ 0  2  5  0  0  0  0  2  0]
 [ 0  0  7  0  0  0  0  1  0]
 [ 0  0  5  5  0  0  0  4  0]
 [ 0  0  1  0  3  0  0  4  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  1 17  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0 11  0]
 [ 3  1  0  0  0  0  0  5  0]]


/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
